# Submission of Calculation with Subsequent Results Query

In this demo, we will submit a calculation, check the status of the job and query the results after it is finished.

In [14]:
from azure.quantum import Workspace
from azure.quantum.job import JobFailedWithResultsError

# insert connection string form Azure Portal Workspace Access Keys
connection_string = ""
workspace = Workspace.from_connection_string(connection_string)

In [15]:
# To submit Accelerated DFT jobs, we will be using the microsoft.dft target in the workspace.
print("Verifying access to Accelerated DFT target.")
target = workspace.get_targets("microsoft.dft")
print("Verification complete.")

Verifying access to Accelerated DFT target.
Verification complete.


Now we submit the calculations to MADFT service.

The QCSchema input below is for a DFT geometry optimization on the molecule phenol (with the geometry given in Bohr)

In [16]:
# Note that for geometry optmimization, the input molecule field is 'initial_molecule'
qcschema_input = {
        "schema_name": "qcschema_optimization_input",
        "schema_version": 1,
        "initial_molecule": {
            "extras": {},
            "symbols": [
                "O",
                "C",
                "C",
                "C",
                "C",
                "C",
                "C",
                "H",
                "H",
                "H",
                "H",
                "H",
                "H"
            ],
            "geometry": [
                4.730542147965709,
                0.034826575331843086,
                0.07810088784463559,
                2.1361232242687977,
                0.017709001458524106,
                0.009088108672780787,
                0.7996954919209014,
                2.290483253979806,
                0.10106814673106823,
                -1.8298562750208616,
                2.2732950799384737,
                -0.04537958079912547,
                -3.1327572801516967,
                -0.00564083248182671,
                -0.28742004920350506,
                -1.790388872477789,
                -2.271959799458856,
                -0.38978844089184156,
                0.8394687277399734,
                -2.2656284043593296,
                -0.24392044354214196,
                5.279447115915874,
                -0.07938333158181043,
                1.8109098053069272,
                1.8583211818406624,
                4.051452964636673,
                0.2691141588512759,
                -2.8675310249318393,
                4.053900197762506,
                0.0241508699472927,
                -5.190440656400895,
                -0.014523603513912258,
                -0.4052054313284032,
                -2.796624853566738,
                -4.060585444078858,
                -0.5909607661605761,
                1.9285725820008635,
                -4.013248220398251,
                -0.3415529925897059
            ]
        },
        "input_specification": {
            "driver": "gradient",
            "model": {
                "method": "m06-2x",
                "basis": "def2-svp"
            }
        }
    }

In [17]:
# Submit jobs:
job_name = 'phenol_go' 
target.submit(input_data=[qcschema_input],name=job_name)

# Query Job Status and Retreive Results

We can retrieve information about a job through [Workspace.get_job](https://learn.microsoft.com/en-us/azure/quantum/optimization-job-management), and also query the results by filtering the job name with [Workspace.list_jobs](https://learn.microsoft.com/en-us/azure/quantum/optimization-job-management).

In [19]:
# query the latest job that match the given name
job = workspace.list_jobs(name_match=job_name)[-1]

# refresh the job SAS for using the API
job.refresh()

# show the status of the job
print(f'Job: "{job_name}" is {job.details.status}')

Job: "phenol_go" is Succeeded


In [20]:
# read the results of the job
if job.details.status == 'Succeeded':
    print(" Job Succeeded ")
    results = job.get_results()
    # QCSchema Output
    qcschema = results["results"][0]
else:
    results = f'"{job_name}" is {job.details.status}...'


 Job Succeeded 


# Results

The results of the calculation are stored in the QCSchema format dict.

We can print the energy of the optimized structure:

In [21]:
# The energy can be accessed:

print("Total Energy of optimized geometry (Hartree): ", qcschema["energies"][-1])

Total Energy of optimized geometry (Hartree):  -307.1041205787441


We can print the coordinates of the optimized geometry. Note that QCSchema output uses Bohr.

In [30]:
import numpy as np
syms = np.array(qcschema["final_molecule"]["symbols"])
coords = np.array(qcschema["final_molecule"]["geometry"] )
NAtoms = len(syms)

print("Geometry in Bohr")
coords_bohr = np.reshape(coords, (-1,3))
print(coords_bohr)
print("")

# Convert coordinates to Angstrom
bohr_to_angstrom = 0.529177
coords_angstrom = coords * bohr_to_angstrom
coords_angstrom = np.reshape(coords_angstrom, (NAtoms,3))
print(f'\n Geometry in Angstrom')
print(coords_angstrom)


Geometry in Bohr
[[ 4.73022902  0.03386118  0.0777041 ]
 [ 2.13574875  0.01746496  0.00889083]
 [ 0.79944694  2.29028248  0.10084778]
 [-1.8301604   2.2730637  -0.04546647]
 [-3.13299875 -0.00582002 -0.28754239]
 [-1.7907981  -2.27225391 -0.39007255]
 [ 0.83898948 -2.26583795 -0.24409322]
 [ 5.27894218 -0.07729315  1.81079002]
 [ 1.85800432  4.05132768  0.26855208]
 [-2.86767161  4.0537677   0.02398202]
 [-5.19069024 -0.01466732 -0.40521116]
 [-2.79714373 -4.06082458 -0.59117622]
 [ 1.92810522 -4.01343886 -0.34167041]]


 Geometry in Angstrom
[[ 2.5031284   0.01791856  0.04111922]
 [ 1.13018912  0.00924206  0.00470482]
 [ 0.42304893  1.21196481  0.05336633]
 [-0.96847879  1.20285303 -0.02405981]
 [-1.65791088 -0.00307982 -0.15216082]
 [-0.94764917 -1.20242451 -0.20641742]
 [ 0.44397394 -1.19902933 -0.12916852]
 [ 2.79349479 -0.04090176  0.95822843]
 [ 0.98321315  2.14386943  0.14211158]
 [-1.51750586  2.14516063  0.01269073]
 [-2.74679389 -0.00776161 -0.21442843]
 [-1.48018413 -2.14889

## Output to QCSchema json file

In [12]:
import json
n = job_name + "_output"
with open(n+".json", "w") as fp:
    json.dump(qcschema, fp, indent=4)